In [1]:
import pandas as pd

In [2]:
SEED = 1

In [3]:
train= pd.read_csv("../data/external/Credit_scoring/train.csv")
test = pd.read_csv("../data/external/Credit_scoring/test.csv")

In [4]:
train.dropna(inplace=True)

In [5]:
train.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [6]:
test.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,NaN,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,2,NaN,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,3,NaN,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,4,NaN,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,5,NaN,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0


In [7]:
X_train = train.drop(columns=["SeriousDlqin2yrs", "Unnamed: 0"], axis = 1)
y_train = train["SeriousDlqin2yrs"]

In [8]:
X_test = test.drop(columns="SeriousDlqin2yrs", axis = 1)
y_test = pd.read_csv("../data/external/Credit_scoring/test-labels.csv")
test = pd.concat([X_test, y_test], axis=1)
test.dropna(inplace=True)
X_test = test.drop(columns=["Probability","Unnamed: 0", "Id"] , axis = 1)
y_test = test["Probability"]

In [9]:
import xgboost

xgb = xgboost.XGBClassifier(
    random_state=SEED, 
    n_jobs = 8,
    n_estimators = 4,
    max_depth = 5

)
xgb.fit(X_train, y_train)

y_pred = xgb.predict(X_test)

In [10]:
import numpy as np

np.mean(y_pred)

0.00957002457002457

In [11]:
y_test_c = (y_test > 0.5).astype(int)

In [12]:
y_test_c

0         0
1         0
2         0
3         0
4         0
         ..
101495    0
101497    0
101498    0
101499    0
101502    0
Name: Probability, Length: 81400, dtype: int64

In [13]:
import dalex as dx

explainer = dx.Explainer(xgb, X_test, y_test_c)

Preparation of a new explainer is initiated

  -> data              : 81400 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 81400 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x124acd940> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0536, mean = 0.0946, max = 0.706
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.58, mean = -0.0779, max = 0.783
  -> model_info        : package xgboost

A new explainer has been created!


In [14]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.521324,0.910141,0.662927,0.991143,0.997032


In [15]:
X_test

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,0.885519,43,0,0.177513,5700.0,4,0,0,0,0.0
1,0.463295,57,0,0.527237,9141.0,15,0,4,0,2.0
2,0.043275,59,0,0.687648,5083.0,12,0,1,0,2.0
3,0.280308,38,1,0.925961,3200.0,7,0,2,0,0.0
4,1.000000,27,0,0.019917,3865.0,4,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...
101495,0.718874,35,1,0.308047,4125.0,8,0,0,1,2.0
101497,0.045230,67,0,0.012198,5000.0,4,0,0,0,0.0
101498,0.282653,24,0,0.068522,1400.0,5,0,0,0,0.0
101499,0.922156,36,3,0.934217,7615.0,8,0,2,0,4.0


In [21]:
# comparision for less than 35 yo and over 35 yo

protected_variable = X_test.age.apply(lambda x: "over" if x > 35 else "under")
privileged_group = "under"

fobject = explainer.model_fairness(
    protected=protected_variable,
    privileged=privileged_group
)

In [22]:
fobject.fairness_check()

Bias detected in 1 metric: TPR

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on 'under'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
           TPR       ACC       PPV  FPR       STP
over  1.568306  1.012232  1.005519  1.0  0.818182


In [23]:
explanation = explainer.model_parts()

In [24]:
explanation.plot()

In [25]:
fobject.plot()


Found NaN's or 0's for models: {'XGBClassifier'}
It is advisable to check 'metric_ratios'


In [ ]:
# WE CAN SEE BIAS TOWARDS PRIVILEGED GROUP

In [28]:
# let's see how the model performs when trained withoud the age column

X_train_no_age, y_train_no_age = X_train.drop(columns = "age"), y_train.drop(columns= "age")
X_test_no_age, y_test_no_age = X_test.drop(columns = "age"), y_test.drop(columns= "age")

In [27]:
xgb_no_age = xgboost.XGBClassifier(
    random_state=SEED, 
    n_jobs = 8,
    n_estimators = 4,
    max_depth = 5
)

xgb_no_age.fit(X_train_no_age, y_train_no_age)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=4, n_jobs=8,
              num_parallel_tree=None, random_state=1, ...)

In [35]:
explainer_no_age = dx.Explainer(xgb_no_age, X_test_no_age, y_test_c)

Preparation of a new explainer is initiated

  -> data              : 81400 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 81400 values
  -> model_class       : xgboost.sklearn.XGBClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x124acd940> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0536, mean = 0.0945, max = 0.701
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.571, mean = -0.0778, max = 0.783
  -> model_info        : package xgboost

A new explainer has been created!


In [43]:
explainer_no_age.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.504412,0.937158,0.655832,0.991155,0.997143


In [44]:
explainer.model_performance()

,recall,precision,f1,accuracy,auc
XGBClassifier,0.521324,0.910141,0.662927,0.991143,0.997032


In [36]:
fobject_no_age = explainer_no_age.model_fairness(protected=protected_variable, privileged=privileged_group)

In [39]:
fobject_no_age.plot()

In [41]:
fobject_no_age.fairness_check()

Bias detected in 1 metric: STP

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on 'under'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
           TPR       ACC       PPV  FPR       STP
over  1.211009  1.010173  1.031694  NaN  0.615385

Warning!
Take into consideration that NaN's are present, consider checking 'metric_scores' plot to see the difference


In [ ]:
# TPR has improved, STP is much  worse

In [46]:
fobject.plot(type = 'radar')

In [47]:
fobject_no_age.plot(type='radar')

Found NaNs in following models: {'XGBClassifier'}
